# Testing Environment

In [16]:
!pip install torch

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 MB 18.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 79.1 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 51.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: /usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip


In [20]:
!pip install --upgrade fair-esm

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: /usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip


In [14]:
!pip install fair-esm==2.0.0

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Defaulting to user installation because normal site-packages is not writeable


In [20]:
# Import packages
import torch
import esm

# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

# Prepare data (first 2 sequences from ESMStructuralSplitDataset superfamily / 4)
data = [
    ("protein1", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"),
    ("protein2", "KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
]

batch_labels, batch_strs, batch_tokens = batch_converter(data)
batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[33], return_contacts=True)
token_representations = results["representations"][33]

# Extract sequence representations
sequence_representations = []
for i, tokens_len in enumerate(batch_lens):
    sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

# Per protein embeddings
protein1_embedding = sequence_representations[0]
protein2_embedding = sequence_representations[1]

In [28]:
test = dict()
test[protein1_embedding] = ""
test[protein2_embedding] = "dasd"

In [29]:
test[protein2_embedding]

'dasd'

In [46]:
import numpy as np
tensor1 = np.array([1,2,3])
tensor2 = np.array([1,2,3])

In [47]:
tensor2 == tensor1

array([ True,  True,  True])

In [41]:
hash(tensor1) == hash(tensor2)

TypeError: unhashable type: 'numpy.ndarray'

In [34]:
hash(tensor1)

4756870704

In [35]:
hash(tensor2)

4757249936

In [36]:
a = dict()
a[tensor1] = 0
a[tensor2] = 1

In [38]:
a[tensor1]

0

In [29]:
sequence_representations = []
for i, tokens_len in enumerate(batch_lens):
    sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

# Look at the unsupervised self-attention map contact predictions
import matplotlib.pyplot as plt
for (_, seq), tokens_len, attention_contacts in zip(data, batch_lens, results["contacts"]):
    plt.matshow(attention_contacts[: tokens_len, : tokens_len])
    plt.title(seq)
    plt.show()

ModuleNotFoundError: No module named 'pyparsing'

In [34]:
test = 5
t = [f"{test}"]
print(t)

['5']


["day154-rep2-IgK-seqC", "day154-rep2-IgK-seqC", "day154-rep2-IgM-seqB", "day154-rep2-IgM-seqB", "day154-rep2-IgG-seqB", "day154-rep2-IgG-seqB", "day154-rep1-IgL-seqC", "day154-rep1-IgL-seqC", "day154-rep2-IgK-seqA", "day154-rep2-IgK-seqA"]

In [9]:
import re

test = ["day154-rep2-IgK-seqC", "day154-rep2-IgK-seqC", 
        "day154-rep2-IgM-seqB", "day154-rep2-IgM-seqB", 
        "day154-rep2-IgG-seqB", "day154-rep2-IgG-seqB", 
        "day154-rep1-IgL-seqC", "day154-rep1-IgL-seqC", 
        "day154-rep2-IgK-seqA", "day154-rep2-IgK-seqA"]

chains = ["IGK", "IGM", "IGG", "IGL"]
dsad = dict()
for rep in test:
    for ch in chains:
        if ch.lower() in rep.lower():
            if not ch in dsad.keys():
                dsad[ch] = list()

            dsad[ch].append(rep)    



In [13]:
len(dsad.values())

4

In [43]:
import re 

text = "What are we aaaa from this message?"

print(re.sub(r"r", '_', text))
print(re.sub(r" h(\w+)", ' _', text))
print(re.sub(r"(\w+)r(\w+)", '_', text))

print(re.match(r"aaaa", text)) # looks only at begining of string
print(re.search(r"aaaa", text))

What a_e we aaaa f_om this message?
What are we aaaa from this message?
What _ we aaaa _ this message?
None
<re.Match object; span=(12, 16), match='aaaa'>


In [5]:
for i in range(10):
    print(i)
    if i == 5:
        break

0
1
2
3
4
5


In [7]:
import sys
sys.stderr.write(f"test{4}")

test4

5